In [335]:
import pandas as pd
import os
import re
from Bio import SeqIO

os.chdir('/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_E104_v1/blastp_on_p')

os.listdir()

files = os.listdir()
outfmt6 = [x for x in files if x.split(".")[-1] == 'top3' and 'anno' in x]
fa_files = [x for x in files if ('protein' in x) and (x.split(".")[-1] == 'fa')]

outfmt6

#The next block should pull in both the initial protein and the blast df.
#The initial protein should become a dataframe that contains proteins sequence name and length.
#This df should be merged with the blast df in a way that proteins without hit should get NA values. 
#Once this is done make two arrays with [p, h], sort this and compare, pull out everything that is identical, and lable it with a new column reverse blast Yes/No.
#Pull out YES and see if they are enriched/depelted in something. NOs need to be checked for high coverage in ph vs h/p mapping and levels of heterozycosity + h on p mapping mappings. 

#read in protein ids for p and h contigs and store names in a list in a dict with unique key id [first part of
#file name].
fa_protein_dict = {}
for file in fa_files:
    seq_list = []
    for seq in SeqIO.parse(open(file), 'fasta'):
        seq_list.append(seq.id)
    key_name = file.split('.')[0]
    fa_protein_dict[key_name] = seq_list

#generate df dict of blast output and filter blast output
header = ['Query', 'Target', 'PctID', 'AlnLgth', 'NumMis', 'NumGap', 'StartQuery', 'StopQuery', 'StartTarget',\
              'StopTarget', 'e-value','BitScore']
outfmt6_dict ={}
#match_dict = {} #get best hits in match_dict[p_protein] = h_protein
hit_df = pd.DataFrame(columns=['p_protein', 'h_protein'])
for outfile in outfmt6:
    key_name =  outfile.split('.')[0]
    df = ''
    df = pd.read_csv(outfile, header = None, names = header, sep='\t')
    df = df[(df['AlnLgth'] > 50) & (df['PctID'] > 50) ] #define paralogous as Aln > 50aa and PctID > 50
    df = df.sort_values(by='BitScore', ascending = True).reset_index(drop=True, inplace=False)
    #df_dup = df[df.duplicated(subset=['Query','PctID', 'BitScore'], keep=False)] #get all hits that have the same bitscore
    print('Dub length %i' % len(df[df.duplicated(subset=['Query', 'BitScore'])]))
    #df = df.drop_duplicates(subset=['Query', 'BitScore'], keep=False) #pull those out
    df = df.drop_duplicates(subset=['Query'], keep ='last') #this pick the best blast hit for each query \
    #in case something slipped through during blast.
    #df = pd.concat([df, df_dup])
    #df = df.drop_duplicates(subset=['Query', 'Target'], keep ='last')
    outfmt6_dict[key_name] = df
    if 'p_ctg' in key_name:
        df['h_protein'] = df['Target']
        df['p_protein'] = df['Query']
    if 'h_ctg' in key_name:
        df['h_protein'] = df['Query']
        df['p_protein'] = df['Target']
    hit_df = pd.concat([hit_df, df.loc[:, ['p_protein', 'h_protein']]])
hit_df = hit_df.sort_values(by='p_protein').reset_index(drop=True)

#duplicates are besties as they are entered twice from both outfmt6
no_bestie_index = hit_df.drop_duplicates(keep = False).index #get the index from non-duplicates

bestie_df = hit_df.drop(no_bestie_index).drop_duplicates()

bestie_df.to_csv(list(outfmt6_dict.keys())[0][:-6] + '.besties.txt', sep='\t', header=None, index=None)
bestie_df['p_protein'].to_csv(list(outfmt6_dict.keys())[0] + '.besties.txt', sep='\t', header=None, index=None)
bestie_df['h_protein'].to_csv(list(outfmt6_dict.keys())[1] + '.besties.txt', sep='\t', header=None, index=None)

no_hits ={}
for key in fa_protein_dict.keys():
    if 'p_' in key:
        no_hits[key] = set(fa_protein_dict[key]) - set(bestie_df['p_protein'])
        pd.DataFrame(list(no_hits[key])).to_csv(key + '.p_proteins.no_besties.txt', sep='\t', header=None, index=None)
    if 'h_' in key:
        no_hits[key] = set(fa_protein_dict[key]) - set(bestie_df['h_protein'])
        pd.DataFrame(list(no_hits[key])).to_csv(key + '.h_proteins.no_besties.txt', sep='\t', header=None, index=None)

_len_pro = 0
_len_out = 0
for x in fa_protein_dict.keys():
    _len_pro += len(fa_protein_dict[x])
    _len_out += len(no_hits[x])
_len_out += len(bestie_df)*2
print(_len_out, _len_pro)
_len_out == _len_pro

Dub length 769
Dub length 449
30270 30270


True

In [3]:
import pandas as pd
import os
import re
from Bio import SeqIO

os.chdir('/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_E104_v1/blastp_on_p')

os.listdir()
hit_df =''
files = os.listdir()
outfmt6 = [x for x in files if x.split(".")[-1] == 'top2' and 'anno' in x]
fa_files = [x for x in files if ('protein' in x) and (x.split(".")[-1] == 'fa')]

outfmt6

#The next block should pull in both the initial protein and the blast df.
#The initial protein should become a dataframe that contains proteins sequence name and length.
#This df should be merged with the blast df in a way that proteins without hit should get NA values. 
#Once this is done make two arrays with [p, h], sort this and compare, pull out everything that is identical, and lable it with a new column reverse blast Yes/No.
#Pull out YES and see if they are enriched/depelted in something. NOs need to be checked for high coverage in ph vs h/p mapping and levels of heterozycosity + h on p mapping mappings. 

#read in protein ids for p and h contigs and store names in a list in a dict with unique key id [first part of
#file name].
fa_protein_dict = {}
fa_protein_length_dict = {}
for file in fa_files:
    seq_list = []
    length_list =[]
    for seq in SeqIO.parse(open(file), 'fasta'):
        seq_list.append(seq.id)
        length_list.append(len(seq.seq))
    key_name = file.split('.')[0]
    fa_protein_dict[key_name] = seq_list
    fa_protein_length_dict[key_name] = dict(zip(seq_list, length_list))

#generate df dict of blast output and filter blast output
header = ['Query', 'Target', 'PctID', 'AlnLgth', 'NumMis', 'NumGap', 'StartQuery', 'StopQuery', 'StartTarget',\
              'StopTarget', 'e-value','BitScore']
outfmt6_dict ={}
#match_dict = {} #get best hits in match_dict[p_protein] = h_protein
hit_df = pd.DataFrame(columns=['p_protein', 'h_protein'])
for outfile in outfmt6:
    key_name =  outfile.split('.')[0]
    df = ''
    df_dup = ''
    df = pd.read_csv(outfile, header = None, names = header, sep='\t')
    #add the query length using to the df using the length dict generated before
    df["QLgth"] = df["Query"].apply(lambda x: fa_protein_length_dict[key_name][x]) 
    df["QCov"] = df['AlnLgth']/df['QLgth']*100 #calculate the % coverage for each querry
    df = df[(df['AlnLgth'] > 50) & (df['PctID'] > 50) ] #define paralogous as Aln > 50aa and PctID > 50
    #this could be more dynamic and the outfmt of blast AlnLngthPct and they greater than 60%
    df = df.sort_values(by='BitScore', ascending = True)
    #print(df.head())
    df_dup = df[df.duplicated(subset=['Query','PctID', 'e-value'], keep=False)] #get all hits that have the same bitscore
    print('Dub length %i' % len(df[df.duplicated(subset=['Query','PctID', 'e-value'])]))
    df = df[~df.duplicated(subset=['Query','PctID', 'e-value'], keep=False)] #pull those out
    df = df.drop_duplicates(subset=['Query'], keep ='last') #this pick the best blast hit for each query \
    #in case something slipped through during blast.
    df = pd.concat([df, df_dup])
    df = df.drop_duplicates(subset=['Query', 'Target'], keep ='last')
    outfmt6_dict[key_name] = df
    if 'p_ctg' in key_name:
        df['h_protein'] = df['Target']
        df['p_protein'] = df['Query']
    if 'h_ctg' in key_name:
        df['h_protein'] = df['Query']
        df['p_protein'] = df['Target']
    hit_df = pd.concat([hit_df, df.loc[:, ['p_protein', 'h_protein']]])
#hit_df = hit_df.sort_values(by='p_protein').reset_index(drop=True)

#duplicates are besties as they are entered twice from both outfmt6
no_bestie_index = hit_df.drop_duplicates(keep = False).index #get the index from non-duplicates

bestie_df = hit_df[hit_df.duplicated(keep='first')]

bestie_df.to_csv(list(outfmt6_dict.keys())[0][:-6] + '.besties.txt', sep='\t', header=None, index=None)
bestie_df['p_protein'].to_csv(list(outfmt6_dict.keys())[0] + '.besties.txt', sep='\t', header=None, index=None)
bestie_df['h_protein'].to_csv(list(outfmt6_dict.keys())[1] + '.besties.txt', sep='\t', header=None, index=None)

no_hits ={}
for key in fa_protein_dict.keys():
    if 'p_' in key:
        no_hits[key] = set(fa_protein_dict[key]) - set(bestie_df['p_protein'])
        pd.DataFrame(list(no_hits[key])).to_csv(key + '.p_proteins.no_besties.txt', sep='\t', header=None, index=None)
    if 'h_' in key:
        no_hits[key] = set(fa_protein_dict[key]) - set(bestie_df['h_protein'])
        pd.DataFrame(list(no_hits[key])).to_csv(key + '.h_proteins.no_besties.txt', sep='\t', header=None, index=None)

_len_pro = 0
_len_out = 0
for x in fa_protein_dict.keys():
    _len_pro += len(fa_protein_dict[x])
    _len_out += len(no_hits[x])
_len_out += (len(bestie_df))*2 - int((bestie_df.groupby(by='p_protein').count() > 1 ).sum()) \
- int((bestie_df.groupby(by='h_protein').count() > 1 ).sum())
_len_out == _len_pro


Dub length 191
Dub length 491


True

In [47]:
import pandas as pd
import os
import re
from Bio import SeqIO

os.chdir('/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_E104_v1/blastp_on_p')

os.listdir()
hit_df =''
files = os.listdir()
outfmt6 = [x for x in files if x.split(".")[-1] == 'alloutfmt6' and 'anno' in x]
fa_files = [x for x in files if ('protein' in x) and (x.split(".")[-1] == 'fa')]

outfmt6

#The next block should pull in both the initial protein and the blast df.
#The initial protein should become a dataframe that contains proteins sequence name and length.
#This df should be merged with the blast df in a way that proteins without hit should get NA values. 
#Once this is done make two arrays with [p, h], sort this and compare, pull out everything that is identical, and lable it with a new column reverse blast Yes/No.
#Pull out YES and see if they are enriched/depelted in something. NOs need to be checked for high coverage in ph vs h/p mapping and levels of heterozycosity + h on p mapping mappings. 

#read in protein ids for p and h contigs and store names in a list in a dict with unique key id [first part of
#file name].
fa_protein_dict = {}
fa_protein_length_dict = {}
for file in fa_files:
    seq_list = []
    length_list =[]
    for seq in SeqIO.parse(open(file), 'fasta'):
        seq_list.append(seq.id)
        length_list.append(len(seq.seq))
    key_name = file.split('.')[0]
    fa_protein_dict[key_name] = seq_list
    fa_protein_length_dict[key_name] = dict(zip(seq_list, length_list))

#generate df dict of blast output and filter blast output
header = ['Query', 'Target', 'PctID', 'AlnLgth', 'NumMis', 'NumGap', 'StartQuery', 'StopQuery', 'StartTarget',\
              'StopTarget', 'e-value','BitScore']
outfmt6_dict ={}
#match_dict = {} #get best hits in match_dict[p_protein] = h_protein
hit_df = pd.DataFrame(columns=['p_protein', 'h_protein'])
for outfile in outfmt6:
    key_name =  outfile.split('.')[0]
    df = ''
    df = pd.read_csv(outfile, header = None, names = header, sep='\t')
    #add the query length using to the df using the length dict generated before
    df["QLgth"] = df["Query"].apply(lambda x: fa_protein_length_dict[key_name][x]) 
    df["QCov"] = df['AlnLgth']/df['QLgth']*100 #calculate the % coverage for each querry
    df = df[(df['QCov'] > 30) & (df['PctID'] > 50) ] #define paralogous as Query coverage > 30% and PctID > 50
    #this could be more dynamic and the outfmt of blast AlnLngthPct and they greater than 60%
    groups = df.groupby(by='Query')
    #now filter the dataframe by the smallest e-value for each group == Query
    df_filtered = groups.apply(lambda g: g[g['e-value'] == g['e-value'].min()]) 
    df_filtered = df_filtered.reset_index(drop=True)
    #in case there is a blast query that hits the same subject twice with the same minimal e-value
    df_filtered = df_filtered.drop_duplicates(subset=['Query', 'Target'], keep ='last')
    outfmt6_dict[key_name] = df_filtered
    if 'p_ctg' in key_name:
        df_filtered['h_protein'] = df_filtered['Target']
        df_filtered['p_protein'] = df_filtered['Query']
    if 'h_ctg' in key_name:
        df_filtered['h_protein'] = df_filtered['Query']
        df_filtered['p_protein'] = df_filtered['Target']
    hit_df = pd.concat([hit_df, df_filtered.loc[:, ['p_protein', 'h_protein']]])

#duplicates are besties as they are entered twice from both outfmt

bestie_df = hit_df[hit_df.duplicated(keep='first')]

bestie_df.to_csv(list(outfmt6_dict.keys())[0][:-6] + '.besties.alloutfmt6.txt', sep='\t', header=None, index=None)
bestie_df['p_protein'].to_csv(list(outfmt6_dict.keys())[0] + '.besties.alloutfmt6.txt', sep='\t', header=None, index=None)
bestie_df['h_protein'].to_csv(list(outfmt6_dict.keys())[1] + '.besties.alloutfmt6.txt', sep='\t', header=None, index=None)

no_hits ={}
for key in fa_protein_dict.keys():
    if 'p_' in key:
        no_hits[key] = set(fa_protein_dict[key]) - set(bestie_df['p_protein'])
        pd.DataFrame(list(no_hits[key])).to_csv(key + '.p_proteins.no_besties.alloutfmt6.txt', sep='\t', header=None, index=None)
    if 'h_' in key:
        no_hits[key] = set(fa_protein_dict[key]) - set(bestie_df['h_protein'])
        pd.DataFrame(list(no_hits[key])).to_csv(key + '.h_proteins.no_besties.alloutfmt6.txt', sep='\t', header=None, index=None)

_len_out = 0
_len_pro = 0
for x in fa_protein_dict.keys():
    _len_pro += len(fa_protein_dict[x])
    print(len(no_hits[x]))
    _len_out += len(no_hits[x])
_len_out += (len(bestie_df))*2 - bestie_df.duplicated(subset="p_protein", keep='last').sum() \
- bestie_df.duplicated(subset="h_protein", keep='last').sum()
_len_out == _len_pro


2974
4809


True

In [48]:
df_filtered.head()

,Query,Target,PctID,AlnLgth,NumMis,NumGap,StartQuery,StopQuery,StartTarget,StopTarget,e-value,BitScore,QLgth,QCov,h_protein,p_protein
0,evm.model.hcontig_000_003.1,evm.model.pcontig_000.2,98.57,140,2,0,1,140,1,140,3.000000e-100,289.0,149,93.959732,evm.model.hcontig_000_003.1,evm.model.pcontig_000.2
1,evm.model.hcontig_000_003.10,evm.model.pcontig_000.9,100.00,57,0,0,1,57,1,57,5.000000e-35,114.0,57,100.000000,evm.model.hcontig_000_003.10,evm.model.pcontig_000.9
2,evm.model.hcontig_000_003.101,evm.model.pcontig_000.96,99.51,405,2,0,1,405,1,405,0.000000e+00,835.0,435,93.103448,evm.model.hcontig_000_003.101,evm.model.pcontig_000.96
3,evm.model.hcontig_000_003.103,evm.model.pcontig_010.34,97.98,247,5,0,4,250,262,508,2.000000e-164,477.0,371,66.576819,evm.model.hcontig_000_003.103,evm.model.pcontig_010.34
4,evm.model.hcontig_000_003.104,evm.model.pcontig_000.97,99.00,400,4,0,1,400,1,400,0.000000e+00,798.0,400,100.000000,evm.model.hcontig_000_003.104,evm.model.pcontig_000.97


In [46]:
_len_out = 0
_len_pro = 0
for x in fa_protein_dict.keys():
    _len_pro += len(fa_protein_dict[x])
    print(len(no_hits[x]))
    _len_out += len(no_hits[x])
_len_out += (len(bestie_df))*2 - bestie_df.duplicated(subset="p_protein", keep='last').sum() \
- bestie_df.duplicated(subset="h_protein", keep='last').sum()
_len_out == _len_pro

2974
4809


True

In [44]:
print(_len_pro)

226211


In [38]:
bestie_df.duplicated(subset="p_protein", keep='last').sum()

3580

In [20]:
int((bestie_df.groupby(by='h_protein').count() > 1 ).sum())

1312

In [6]:
groups = df.groupby(by='Query')
df_test = groups.apply(lambda g: g[g['BitScore'] == g['BitScore'].max()])
#this should fix it if we have multipe hits. First transform the data frame by filtering 
#that has not the min e-value for each group. afterwards do the aggregation and filtering.
#also add a query length column and pctlengthalign for more dynamic filtering

In [7]:
df_test['Query'] = df_test.index.get_level_values(0)

In [8]:
df_test = df_test.reset_index(drop=True)

In [9]:
len(df[df.duplicated(subset=['Query', 'BitScore'])])

979

In [10]:
df_test.duplicated(subset='Query', keep=False).sum()

453

In [11]:
df_test

,Query,Target,PctID,AlnLgth,NumMis,NumGap,StartQuery,StopQuery,StartTarget,StopTarget,e-value,BitScore,QLgth,QCov,h_protein,p_protein
0,evm.model.hcontig_000_003.1,evm.model.pcontig_000.2,98.57,140,2,0,1,140,1,140,3.000000e-100,289.0,149,93.959732,evm.model.hcontig_000_003.1,evm.model.pcontig_000.2
1,evm.model.hcontig_000_003.10,evm.model.pcontig_000.9,100.00,57,0,0,1,57,1,57,5.000000e-35,114.0,57,100.000000,evm.model.hcontig_000_003.10,evm.model.pcontig_000.9
2,evm.model.hcontig_000_003.101,evm.model.pcontig_000.96,99.51,405,2,0,1,405,1,405,0.000000e+00,835.0,435,93.103448,evm.model.hcontig_000_003.101,evm.model.pcontig_000.96
3,evm.model.hcontig_000_003.103,evm.model.pcontig_010.34,97.98,247,5,0,4,250,262,508,2.000000e-164,477.0,371,66.576819,evm.model.hcontig_000_003.103,evm.model.pcontig_010.34
4,evm.model.hcontig_000_003.104,evm.model.pcontig_000.97,99.00,400,4,0,1,400,1,400,0.000000e+00,798.0,400,100.000000,evm.model.hcontig_000_003.104,evm.model.pcontig_000.97
5,evm.model.hcontig_000_003.105,evm.model.pcontig_000.98,99.55,220,1,0,1,220,1,220,2.000000e-158,441.0,220,100.000000,evm.model.hcontig_000_003.105,evm.model.pcontig_000.98
6,evm.model.hcontig_000_003.106,evm.model.pcontig_000.99,98.56,347,5,0,1,347,6,352,0.000000e+00,650.0,347,100.000000,evm.model.hcontig_000_003.106,evm.model.pcontig_000.99
7,evm.model.hcontig_000_003.107,evm.model.pcontig_000.100,99.21,1141,9,0,1,1141,1,1141,0.000000e+00,2330.0,1163,98.108340,evm.model.hcontig_000_003.107,evm.model.pcontig_000.100
8,evm.model.hcontig_000_003.108,evm.model.pcontig_000.101,89.03,237,20,1,1,237,1,231,2.000000e-150,421.0,237,100.000000,evm.model.hcontig_000_003.108,evm.model.pcontig_000.101
9,evm.model.hcontig_000_003.109,evm.model.pcontig_000.102,100.00,107,0,0,1,107,1,107,8.000000e-73,214.0,107,100.000000,evm.model.hcontig_000_003.109,evm.model.pcontig_000.102


In [390]:
(df_test.groupby('Query').count() > 1).sum()

Target         457
PctID          457
AlnLgth        457
NumMis         457
NumGap         457
StartQuery     457
StopQuery      457
StartTarget    457
StopTarget     457
e-value        457
BitScore       457
h_protein      457
p_protein      457
dtype: int64

In [351]:
groups = df.groupby(by='Query')

print(groups.apply(lambda g: g[g['B'] == g['B'].max()]))

TypeError: invalid type comparison

In [343]:
hit_df[hit_df['p_protein'] == 'evm.model.pcontig_017.248']

,p_protein,h_protein
11837,evm.model.pcontig_017.248,evm.model.hcontig_018_016.15
14805,evm.model.pcontig_017.248,evm.model.hcontig_018_016.15


In [334]:
len(bestie_df)

10908

In [ ]:
#one problem might occur if the you have multiple best blast hits and you drop the wrong one
#example to look at is evm.model.pcontig_225.7 evm.model.hcontig_225_001.8
#write out proteins with blast hits that weren't reciprocal besties. These can be used to eliminate good nt blast hits
#if they hit something better elsewhere and not the corresponding position identified by blast.
#no bestie -> get gene sequence and blast against other haplotype -> convert bed -> see if overlapping gene feature 
#if so why no blastp hit? -> if no gene it check for expression using bam file -> if no expression all good 
#-> if expression check for frame shit and such

# All done after hear

In [13]:
!head Pst_E104_v1_p_ctg.besties.txt

evm.model.pcontig_000.100
evm.model.pcontig_000.101
evm.model.pcontig_000.102
evm.model.pcontig_000.103
evm.model.pcontig_000.104
evm.model.pcontig_000.105
evm.model.pcontig_000.108
evm.model.pcontig_000.11
evm.model.pcontig_000.110
evm.model.pcontig_000.111


In [14]:
annotation_df_p = pd.read_csv('/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/122016_assembly/annotations/annotations.21122016.Pst_E104_p_ctg_v1.RepaseTPSI_filtered.txt', sep='\t', header =None )

annotation_df_p_eff = annotation_df_p[annotation_df_p[2] == 'EffectorP']     

(len(fa_protein_dict['Pst_E104_v1_p_ctg']) - len(bestie_df))/len(fa_protein_dict['Pst_E104_v1_p_ctg'])

0.3447238071352436

In [16]:
len(annotation_df_p_eff[annotation_df_p_eff[0].isin(no_hits['Pst_E104_v1_p_ctg'])])/len(annotation_df_p_eff)

0.2624113475177305

In [17]:
annotation_df_h = pd.read_csv('/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/122016_assembly/annotations/annotations.21122016.Pst_E104_h_ctg_v1.RepaseTPSI_filtered.txt', sep='\t', header =None )

annotation_df_h_eff = annotation_df_h[annotation_df_h[2] == 'EffectorP']     

(len(fa_protein_dict['Pst_E104_v1_h_ctg']) - len(bestie_df))/len(fa_protein_dict['Pst_E104_v1_h_ctg'])

0.2702325256616158

In [18]:
len(annotation_df_h_eff[annotation_df_h_eff[0].isin(no_hits['Pst_E104_v1_h_ctg'])])/len(annotation_df_h_eff)

0.18848758465011287

In [19]:
bestie_df_outfmt6 = outfmt6_dict['Pst_E104_v1_p_ctg'][outfmt6_dict['Pst_E104_v1_p_ctg']['Query'].isin(bestie_df['p_protein'])]

In [20]:
bestie_df_outfmt6.describe()

,PctID,AlnLgth,NumMis,NumGap,StartQuery,StopQuery,StartTarget,StopTarget,e-value,BitScore
count,10451.000000,10451.000000,10451.000000,10451.000000,10451.000000,10451.000000,10451.000000,10451.000000,1.045100e+04,10451.000000
mean,97.712761,416.717156,6.544828,0.274902,10.136829,424.913118,8.367333,423.099129,9.568673e-10,814.767056
std,5.133646,347.919220,15.735413,0.828165,61.087489,351.490351,52.695005,350.633306,7.449378e-08,690.610911
min,50.860000,51.000000,0.000000,0.000000,1.000000,51.000000,1.000000,51.000000,0.000000e+00,40.000000
25%,97.830000,176.000000,0.000000,0.000000,1.000000,183.000000,1.000000,182.000000,0.000000e+00,338.000000
50%,99.520000,330.000000,1.000000,0.000000,1.000000,338.000000,1.000000,337.000000,0.000000e+00,639.000000
75%,100.000000,539.000000,6.000000,0.000000,1.000000,548.000000,1.000000,545.000000,2.000000e-119,1057.500000
max,100.000000,5230.000000,410.000000,16.000000,1886.000000,5230.000000,1883.000000,5230.000000,7.000000e-06,10748.000000


In [21]:
bestie_df_outfmt6[bestie_df_outfmt6['Query'].isin(annotation_df_p_eff[0])].describe()

,PctID,AlnLgth,NumMis,NumGap,StartQuery,StopQuery,StartTarget,StopTarget,e-value,BitScore
count,728.000000,728.000000,728.000000,728.000000,728.000000,728.000000,728.000000,728.000000,7.280000e+02,728.000000
mean,97.992363,154.637363,2.277473,0.094780,2.222527,155.354396,1.943681,155.192308,9.615386e-27,305.438874
std,4.717109,58.026393,4.951477,0.423615,9.253123,57.562269,7.152754,58.089637,2.594373e-25,113.010931
min,53.240000,51.000000,0.000000,0.000000,1.000000,51.000000,1.000000,52.000000,0.000000e+00,85.500000
25%,98.010000,111.000000,0.000000,0.000000,1.000000,111.000000,1.000000,111.000000,1.575000e-136,223.000000
50%,100.000000,140.000000,0.000000,0.000000,1.000000,141.000000,1.000000,140.000000,2.000000e-95,274.500000
75%,100.000000,196.000000,2.000000,0.000000,1.000000,197.000000,1.000000,197.000000,2.000000e-76,382.500000
max,100.000000,486.000000,55.000000,7.000000,195.000000,486.000000,88.000000,486.000000,7.000000e-24,842.000000


In [22]:
expression_folder = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_E104_v1/Pst_E104_v1_ph_ctg/stringtie_e/first_round_8000/'
e_header = ["gene_id","logFC","logCPM","PValue","FDR"]
HE_vs_UG_df = pd.read_csv(expression_folder+'de_UG_HE_no_IT0_8000.csv', skiprows =1, names=e_header, header = None)

In [23]:
HE_vs_UG_df_top_1000 = HE_vs_UG_df[(HE_vs_UG_df['PValue'] < 0.01) & (HE_vs_UG_df['FDR'] < 0.01) & (HE_vs_UG_df['logFC'] > 2)].sort_values(by='logFC', ascending =False)

In [24]:
HE_vs_UG_df['protein_id'] = HE_vs_UG_df['gene_id'].apply(lambda x: 'evm.model'+x[6:])

In [25]:
HE_vs_UG_df_top_1000['protein_id'] = HE_vs_UG_df_top_1000['gene_id'].apply(lambda x: 'evm.model'+x[6:])

In [26]:
bestie_p_HE_vs_UG_df_top_1000_index = bestie_df[bestie_df['p_protein'].isin(HE_vs_UG_df_top_1000['protein_id'])].index
bestie_h_HE_vs_UG_df_top_1000_index = bestie_df[bestie_df['h_protein'].isin(HE_vs_UG_df_top_1000['protein_id'])].index
print(len(bestie_h_HE_vs_UG_df_top_1000_index))
print(len(bestie_p_HE_vs_UG_df_top_1000_index))

1468
1490


In [27]:
dif_p_vs_h = bestie_p_HE_vs_UG_df_top_1000_index.difference(bestie_h_HE_vs_UG_df_top_1000_index)
dif_h_vs_p = bestie_h_HE_vs_UG_df_top_1000_index.difference(bestie_p_HE_vs_UG_df_top_1000_index)

In [28]:
HE_vs_UG_df_top_1000[(HE_vs_UG_df_top_1000['protein_id'].isin(bestie_df['h_protein'][dif_h_vs_p]))]

,gene_id,logFC,logCPM,PValue,FDR,protein_id
860,evm.TU.hcontig_020_021.80,13.436300,4.062621,5.367086e-49,1.136875e-47,evm.model.hcontig_020_021.80
6177,evm.TU.hcontig_034_001.60,12.962684,3.379426,7.444779e-12,2.197764e-11,evm.model.hcontig_034_001.60
5218,evm.TU.hcontig_000_050.93,12.872342,3.341580,2.639333e-14,9.223252e-14,evm.model.hcontig_000_050.93
3503,evm.TU.hcontig_031_007.6,11.443336,2.042424,2.162724e-20,1.125678e-19,evm.model.hcontig_031_007.6
4087,evm.TU.hcontig_001_180.6,11.156688,2.031067,5.163008e-18,2.303399e-17,evm.model.hcontig_001_180.6
4124,evm.TU.hcontig_031_005.114,10.948043,1.587420,7.005742e-18,3.097472e-17,evm.model.hcontig_031_005.114
7296,evm.TU.hcontig_029_010.84,10.676368,5.423470,1.513494e-09,3.782803e-09,evm.model.hcontig_029_010.84
2978,evm.TU.hcontig_016_024.76,10.364506,0.968792,5.373471e-23,3.289740e-22,evm.model.hcontig_016_024.76
3680,evm.TU.hcontig_004_018.2,10.032872,0.598136,1.538400e-19,7.622205e-19,evm.model.hcontig_004_018.2
7984,evm.TU.hcontig_007_006.202,10.000667,6.398951,2.367696e-08,5.407895e-08,evm.model.hcontig_007_006.202


In [29]:
HE_vs_UG_df[HE_vs_UG_df['protein_id'].isin(bestie_df['p_protein'][dif_h_vs_p])].head()

,gene_id,logFC,logCPM,PValue,FDR,protein_id
2079,evm.TU.pcontig_000.410,1.986678,5.348733,1.455062e-29,1.275838e-28,evm.model.pcontig_000.410
2594,evm.TU.pcontig_006.96,1.895500,5.424008,1.709167e-25,1.201225e-24,evm.model.pcontig_006.96
2637,evm.TU.pcontig_002.549,1.850649,5.044498,3.522789e-25,2.435505e-24,evm.model.pcontig_002.549
2786,evm.TU.pcontig_044.45,1.676377,5.142921,3.685549e-24,2.411806e-23,evm.model.pcontig_044.45
2917,evm.TU.pcontig_006.415,1.714764,5.278586,2.380898e-23,1.488102e-22,evm.model.pcontig_006.415


In [30]:
diff_expressed = []

In [31]:
not_expressed_p = bestie_df['p_protein'][dif_h_vs_p][~bestie_df['p_protein'][dif_h_vs_p].isin(HE_vs_UG_df['protein_id'])]

In [32]:
not_expressed_h = bestie_df['h_protein'][dif_p_vs_h][~bestie_df['h_protein'][dif_p_vs_h].isin(HE_vs_UG_df['protein_id'])]

In [33]:
print((len(not_expressed_h) + len(not_expressed_p)))

209


In [34]:
print((len(not_expressed_h) + len(not_expressed_p))/len(HE_vs_UG_df_top_1000))

0.05461196759864123


In [35]:
not_expressed_besties_w_bestie = list(not_expressed_h) + list(not_expressed_p)
expressed_besties_wo_bestie = list(bestie_df[bestie_df['h_protein'].isin(not_expressed_h)]['p_protein'])
expressed_besties_wo_bestie += list(bestie_df[bestie_df['p_protein'].isin(not_expressed_p)]['h_protein'])
len(not_expressed_besties_w_bestie) == len(expressed_besties_wo_bestie)

True

In [36]:
gff = pd.read_csv('/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/122016_assembly/Pst_E104_v1_ph_ctg.anno.RepaseTPSI_filtered.gff3', header=None,  sep='\t')

In [75]:
len(bestie_df)

10451

In [78]:
len(fa_protein_dict['Pst_E104_v1_h_ctg'])

14321

In [37]:
gff.head()

,0,1,2,3,4,5,6,7,8
0,pcontig_241,EVM,gene,13967,14266,.,+,.,ID=evm.TU.pcontig_241.2;Name=EVM%20prediction%...
1,pcontig_241,EVM,mRNA,13967,14266,.,+,.,ID=evm.model.pcontig_241.2;Parent=evm.TU.pcont...
2,pcontig_241,EVM,exon,13967,14266,.,+,.,ID=evm.model.pcontig_241.2.exon1;Parent=evm.mo...
3,pcontig_241,EVM,CDS,13967,14266,.,+,0,ID=cds.evm.model.pcontig_241.2;Parent=evm.mode...
4,pcontig_241,EVM,gene,4229,4450,.,-,.,ID=evm.TU.pcontig_241.1;Name=EVM%20prediction%...


In [40]:
gff['ID'] = gff[8].apply(col_8_id)

In [39]:
def col_8_id(x):
    import re
    pattern = r'ID=([a-zA-Z0-9_.]*);'
    regex = re.compile(pattern)  
    m = regex.search(x)
    return m.groups()[0]

In [74]:
len(HE_vs_UG_df_top_1000)

3827

In [41]:
len(gff[gff['ID'].isin(not_expressed_besties_w_bestie)])

209

In [42]:
gff[gff['ID'].isin(not_expressed_besties_w_bestie)][8].str.contains('EffectorP').sum()

24

In [44]:
len(gff[gff['ID'].isin(expressed_besties_wo_bestie)])

209

In [45]:
gff[gff['ID'].isin(expressed_besties_wo_bestie)][8].str.contains('EffectorP').sum()

25

In [46]:
gff[8].str.contains('EffectorP').sum()/len(gff[gff[2] == 'mRNA'])

0.061876445325404691

In [49]:
gff[8].str.contains('EffectorP').sum()/len(gff[gff[2] == 'mRNA'])

0.061876445325404691

In [50]:
gff[8].str.contains('SIGNALP_EUK:SignalP-noTM').sum()/len(gff[gff[2] == 'mRNA'])

0.14142715559960356

In [51]:
gff[gff['ID'].isin(expressed_besties_wo_bestie)][8].str.contains('SIGNALP_EUK:SignalP-noTM').sum()

62

In [52]:
gff[gff['ID'].isin(not_expressed_besties_w_bestie)][8].str.contains('SIGNALP_EUK:SignalP-noTM').sum()

63

In [58]:
gff[gff['ID'].isin(HE_vs_UG_df_top_1000['protein_id'])][8].str.contains('EffectorP').sum()/gff[8].str.contains('EffectorP').sum()

0.18846769887880405

In [59]:
gff[8].str.contains('EffectorP').sum()/len(gff[gff[2] == 'mRNA'])

0.061876445325404691

In [60]:
gff[gff['ID'].isin(HE_vs_UG_df_top_1000['protein_id'])][8].str.contains('SIGNALP_EUK:SignalP-noTM').sum()/gff[8].str.contains('SIGNALP_EUK:SignalP-noTM').sum()

0.26489138051857042

In [61]:
gff[8].str.contains('SIGNALP_EUK:SignalP-noTM').sum()/len(gff[gff[2] == 'mRNA'])

0.14142715559960356

In [121]:
bestie_df[bestie_df['p_protein'].isin(HE_vs_UG_df['protein_id'])]

,p_protein,h_protein
19,evm.model.pcontig_000.110,evm.model.hcontig_000_003.118
21,evm.model.pcontig_000.111,evm.model.hcontig_000_003.119
28,evm.model.pcontig_000.116,evm.model.hcontig_000_003.122
54,evm.model.pcontig_000.136,evm.model.hcontig_000_003.150
56,evm.model.pcontig_000.137,evm.model.hcontig_000_003.151
58,evm.model.pcontig_000.138,evm.model.hcontig_000_003.152
62,evm.model.pcontig_000.14,evm.model.hcontig_000_003.15
64,evm.model.pcontig_000.140,evm.model.hcontig_000_003.154
66,evm.model.pcontig_000.141,evm.model.hcontig_000_003.155
74,evm.model.pcontig_000.145,evm.model.hcontig_000_003.159


In [66]:
gff['anno'] = gff[gff['ID'].isin(HE_vs_UG_df_top_1000['protein_id'])][8].apply(lambda x: x.split(';')[-1])

In [69]:
gff[gff['ID'].isin(HE_vs_UG_df_top_1000['protein_id'])]['anno'].str.contains('InterPro').sum()

1461

In [70]:
len(gff[gff['ID'].isin(HE_vs_UG_df_top_1000['protein_id'])]['anno'])

3827

In [72]:
gff['anno'] = gff[8].apply(lambda x: x.split(';')[-1])

In [73]:
gff['anno'].str.contains('InterPro').sum()

12748

In [165]:
HE_vs_UG_df[HE_vs_UG_df['protein_id'].isin(bestie_df['p_protein'][dif_h_vs_p])]

,gene_id,logFC,logCPM,PValue,FDR,protein_id
2079,evm.TU.pcontig_000.410,1.986678,5.348733,1.455062e-29,1.275838e-28,evm.model.pcontig_000.410
2594,evm.TU.pcontig_006.96,1.895500,5.424008,1.709167e-25,1.201225e-24,evm.model.pcontig_006.96
2637,evm.TU.pcontig_002.549,1.850649,5.044498,3.522789e-25,2.435505e-24,evm.model.pcontig_002.549
2786,evm.TU.pcontig_044.45,1.676377,5.142921,3.685549e-24,2.411806e-23,evm.model.pcontig_044.45
2917,evm.TU.pcontig_006.415,1.714764,5.278586,2.380898e-23,1.488102e-22,evm.model.pcontig_006.415
2947,evm.TU.pcontig_000.376,1.894629,4.853953,3.668937e-23,2.269812e-22,evm.model.pcontig_000.376
3118,evm.TU.pcontig_052.120,1.899595,6.011158,2.608933e-22,1.525544e-21,evm.model.pcontig_052.120
3153,evm.TU.pcontig_058.8,1.806411,6.227503,3.908519e-22,2.260101e-21,evm.model.pcontig_058.8
3377,evm.TU.pcontig_091.43,1.782711,6.952537,5.736237e-21,3.097025e-20,evm.model.pcontig_091.43
3575,evm.TU.pcontig_030.32,1.642079,5.630258,4.571772e-20,2.331655e-19,evm.model.pcontig_030.32


In [2]:
import pandas as pd
import os

In [13]:
def col_8_id(x):
    import re
    pattern = r'ID=([a-zA-Z0-9_.]*);'
    regex = re.compile(pattern)  
    m = regex.search(x)
    return m.groups()[0]

In [8]:
folder = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/122016_assembly/'

In [6]:
gene_gff = [x for x in os.listdir('/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/122016_assembly/') if 'gene' in x and 'gff3' in x]

In [16]:
for gff in gene_gff:
    gene = ''
    gene_df = pd.read_csv(folder + gff, header = None, sep='\t' )
    gene_df[2] = gene_df[8].apply(col_8_id)
    gene_df.to_csv(folder+gff, header=None, sep='\t', index=None)

In [17]:
gene_df.head()

,0,1,2,3,4,5,6,7,8
0,hcontig_014_192,EVM,evm.TU.hcontig_014_192.6,26367,27785,.,+,.,ID=evm.TU.hcontig_014_192.6;Name=EVM%20predict...
1,hcontig_014_192,EVM,evm.TU.hcontig_014_192.4,20324,21477,.,+,.,ID=evm.TU.hcontig_014_192.4;Name=EVM%20predict...
2,hcontig_014_192,EVM,evm.TU.hcontig_014_192.5,21893,25780,.,+,.,ID=evm.TU.hcontig_014_192.5;Name=EVM%20predict...
3,hcontig_014_192,EVM,evm.TU.hcontig_014_192.3,16637,17518,.,-,.,ID=evm.TU.hcontig_014_192.3;Name=EVM%20predict...
4,hcontig_014_192,EVM,evm.TU.hcontig_014_192.8,33268,33603,.,+,.,ID=evm.TU.hcontig_014_192.8;Name=EVM%20predict...


In [22]:
os.chdir('/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_E104_v1/blastp_on_p')

In [86]:
!makeblastdb -in Pst_E104_v1_p_ctg.fa -dbtype nucl



Building a new DB, current time: 01/25/2017 14:29:36
New DB name:   /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_E104_v1/blastp_on_p/Pst_E104_v1_p_ctg.fa
New DB title:  Pst_E104_v1_p_ctg.fa
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_E104_v1/blastp_on_p/Pst_E104_v1_p_ctg.fa
Keep Linkouts: T
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 176 sequences in 1.0142 seconds.


In [85]:
!makeblastdb -in Pst_E104_v1_h_ctg.fa -dbtype nucl



Building a new DB, current time: 01/25/2017 14:29:31
New DB name:   /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_E104_v1/blastp_on_p/Pst_E104_v1_h_ctg.fa
New DB title:  Pst_E104_v1_h_ctg.fa
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_E104_v1/blastp_on_p/Pst_E104_v1_h_ctg.fa
Keep Linkouts: T
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 480 sequences in 0.909299 seconds.


In [ ]:
from Bio import SeqIO

folder_p = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_E104_v1/blastp_on_p/'

no_besties = [x for x in os.listdir(folder_p) if 'no_besties.txt' in x]
gene_files = [x for x in os.listdir(folder_p) if '.gene.' in x and '.fa' in x]
no_besties.sort()
gene_files.sort()

#simply pulls in the gene sequences of missing besties
for no_b, gene_file in zip(no_besties, gene_files):
    no_bestie_list = pd.read_csv(folder_p+no_b, header=None, sep='\t')[0].tolist()
    no_bestie_list = [x.replace('evm.model', 'evm.TU') for x in no_bestie_list]
    no_bestie_seq = []
    for seq in SeqIO.parse(open(folder_p + gene_file), 'fasta'):
        if seq.id in no_bestie_list:
            no_bestie_seq.append(seq)
    out_f = folder_p + no_b[:-3].replace('protein', 'gene') + 'fa'
    f_handle = open(out_f,'w') #need to generate handle for writing and
    SeqIO.write(no_bestie_seq, f_handle, 'fasta')
    f_handle.close() #closing file afterwards again

gene_files_no_besties = [x for x in os.listdir(folder_p) if 'gene' in x and '.fa' in x and 'besties.fa' in x and 'outfmt6' not in x]
blast_db_nt = [x for x in os.listdir(folder_p) if '_ctg.fa' in x and x[-3:] == '.fa']
gene_files_no_besties.sort()
blast_db_nt.sort()

os.chdir(folder_p)

!blastn -db {blast_db_nt[1]} -query {gene_files_no_besties[0]} -max_target_seqs 1 -outfmt 6 -evalue 1e-5 -num_threads 1 \
> {gene_files_no_besties[0]}.outfmt6


in_file =gene_files_no_besties[0]+'.outfmt6'
blast_outfmt6_to_bed(in_file)

!blastn -db {blast_db_nt[0]} -query {gene_files_no_besties[1]} -max_target_seqs 1 -outfmt 6 -evalue 1e-5 -num_threads 1 \
> {gene_files_no_besties[1]}.outfmt6


in_file =gene_files_no_besties[1]+'.outfmt6'
blast_outfmt6_to_bed(in_file)

In [204]:
!head -30 Pst_E104_v1_p_ctg.p_genes.no_besties.fa.outfmt6

evm.TU.pcontig_241.2	hcontig_241_001	100.00	300	0	0	1	300	6754	7053	2e-157	555
evm.TU.pcontig_241.1	hcontig_238_001	100.00	222	0	0	1	222	17244	17023	3e-114	411
evm.TU.pcontig_225.7	hcontig_225_001	100.00	357	0	0	1	357	24891	25247	0.0	660
evm.TU.pcontig_225.1	hcontig_086_002	98.36	244	1	3	1	243	85796	86037	1e-118	425
evm.TU.pcontig_225.1	hcontig_086_002	97.96	245	2	3	1	243	78552	78795	1e-117	422
evm.TU.pcontig_225.2	hcontig_086_002	98.68	227	2	1	1	226	79146	79372	2e-111	401
evm.TU.pcontig_225.2	hcontig_086_002	98.68	227	2	1	1	226	86388	86614	2e-111	401
evm.TU.pcontig_234.1	hcontig_238_002	100.00	288	0	0	1	288	2062	2349	7e-151	532
evm.TU.pcontig_164.3	hcontig_037_011	90.91	242	22	0	4	245	179738	179979	2e-88	326
evm.TU.pcontig_164.1	hcontig_004_076	90.10	303	29	1	1	303	22199	21898	1e-108	392
evm.TU.pcontig_166.1	hcontig_007_007	80.26	390	59	10	1	372	221499	221110	5e-74	278
evm.TU.pcontig_166.11	hcontig_031_003	100.00	1379	0	0	1	1379	13619	14997	0.0	2547
evm.TU.pcontig_166.10	hcontig_166_0

In [155]:
def blast_outfmt6_to_bed(x):
    blast_fo = open(x, 'r')
    blast_lines = blast_fo.readlines()
    bed_file_name = x + '.bed'
    bed_fo = open(bed_file_name, 'w+')
    for l in blast_lines:
        content = l.split('\t')
        if int(content[8]) - int(content[9]) < 1:
            print(content[1], int(content[8]) -1, content[9], content[0], content[10], "+", sep="\t", file=bed_fo) 
        else:
            print(content[1], int(content[9]) -1, content[8],  content[0], content[10], "-", sep = "\t", file=bed_fo)
    blast_fo.close()
    bed_fo.close()